In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
    import procurement; reload(procurement)
reload_all()
from itables import show

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

429 Client Error: Too Many Requests for url: https://pricing-api.tradeskillmaster.com/region/13


In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from blizzard import collapse_languages
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [6]:
from cytoolz import groupby
import pandas as pd

df_sales = pd.read_csv("Accounting_Earthfury_sales.csv")
df_sales["mode"] = "sold"
df_purchases = pd.read_csv("Accounting_Earthfury_purchases.csv")
df_purchases["mode"] = "purchased"
df_expired = pd.read_csv("Accounting_Earthfury_expired.csv")
df_expired["mode"] = "expired"
df_canceled = pd.read_csv("Accounting_Earthfury_canceled.csv")
df_canceled["mode"] = "canceled"
df_all = pd.concat([df_sales, df_purchases, df_expired, df_canceled])

entries = list(df_all.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {
    k: sum(
        (
            vv["price"]*vv["quantity"] if vv["mode"] == "sold" else
            -vv["price"]*vv["quantity"] if vv["mode"] == "purchased" else
            0
        ) for vv in v
    )
    for (k, v) in grouped.items()
}
ah_events = {k: [vv["mode"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sold") / (v.count("sold") + v.count("expired") + v.count("canceled"))
    for (k, v) in ah_events.items()
    if v.count("sold") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
show(pd.DataFrame(sorted(
    [v for v in agg.values() if v["num"] > 4],
    key=lambda entry: entry["net"],
    reverse=True,
)))

<ipython-input-6-e5b0eba07282>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  entries = list(df_all.transpose().to_dict().values())


In [7]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
chaotic skyflare diamond
skyflare diamond
potent monarch topaz
monarch topaz
runed scarlet ruby
scarlet ruby
purified twilight opal
twilight opal
potion of speed
potion of wild magic
lichbloom
frostweave cloth
infinite dust
greater cosmic essence
dream shard
eternal belt buckle
titansteel bar
titansteel spellblade
mageweave cloth
rough stone
copper bar
coarse stone
linen cloth
bronze bar
heavy stone
moss agate
wool cloth
medium leather
steel bar
solid stone
mithril bar
mageweave cloth
dense stone
thorium bar
runecloth
fel iron bar
mote of earth
mote of fire
netherweave cloth
adamantite bar
cobalt bar
frostweave cloth
crystallized water
crystallized earth
crystallized fire
borean leather
eternal shadow
saronite bar
liferoot
blackmouth oil
briarthorn
bruiseweed
elixir of greater defense
fadeleaf
goldclover
goldthorn
greater healing potion
imbued vial
kingsblood
lesser invisibility potion
lichbloom
liferoot
major troll's blood elixir
nature protection potion
oily blackmouth
runic mana potion
stranglekelp
swiftness potion
swiftthistle
swim speed potion
wild steelbloom
elixir of water walking
elixir of mighty thoughts
""".strip().splitlines()

desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

In [8]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [9]:
def item_infos(items):
    return pd.DataFrame([item_info(i) for i in items])

In [10]:
ii = item_infos(_names)
show(ii)

In [11]:
from crafting import Recipes
from crafting import purchase_price
from crafting import crafting_price
from crafting import gold


r = Recipes(items)
i = r.ingredients

def buy(item, price_method=gold("realm_market_value")):
    return purchase_price(item_info, item, price_method)


def buy_now(item):
    return buy(item, price_method=gold("min"))


def buy_market(item):
    return buy(item, price_method=gold("realm_market_value"))


recipe_listing = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

eternal fire
10 crystallized fire

eternal air
10 crystallized air

eternal life
10 crystallized life

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence

10 crystallized shadow
eternal shadow

10 crystallized water
eternal water

10 crystallized fire
eternal fire

10 crystallized air
eternal air

10 crystallized life
eternal life

3 lesser planar essence
greater planar essence

3 lesser cosmic essence
greater cosmic essence

2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial

titansteel spellblade
6 titansteel bar + 6 infinite dust + 2 frozen orb

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

glyph of wild growth
resilient parchment + ethereal ink

spellweave robe
8 spellweave + 6 bolt of imbued frostweave + eternium thread + frozen orb

spellweave
bolt of imbued frostweave + 2 eternal fire

bolt of imbued frostweave
2 bolt of frostweave + 2 infinite dust

bolt of frostweave
5 frostweave cloth

earthsiege diamond
dark jade + huge citrine + eternal fire

skyflare diamond
bloodstone + chalcedony + eternal air

noise machine
2 froststeel tube + 2 overcharged capacitor + 8 handful of cobalt bolts

froststeel tube
8 cobalt bar + crystallized water

overcharged capacitor
4 cobalt bar + crystallized earth

handful of cobalt bolts
2 cobalt bar

cobalt bar
cobalt ore

mechanized snow goggles
8 saronite bar + 2 borean leather + eternal shadow

saronite bar
2 saronite ore

wormhole generator: northrend
8 titanium bar + 2 eternal shadow + 2 eternal fire + 2 eternal water + 2 eternal air

pygmy oil
pygmy suckerfish

runic mana potion
2 lichbloom + goldclover + imbued vial

greater healing potion
liferoot + kingsblood + leaded vial

major troll's blood elixir
liferoot + bruiseweed + leaded vial

nature protection potion
liferoot + stranglekelp + leaded vial

swiftness potion
swiftthistle + briarthorn + leaded vial

swim speed potion
swiftthistle + blackmouth oil + leaded vial

blackmouth oil
2 oily blackmouth

elixir of greater defense
wild steelbloom + goldthorn + leaded vial

lesser invisibility potion
wild steelbloom + fadeleaf + leaded vial

great feast
chunk o' mammoth + shoveltusk flank + worm meat + 2 chilled meat

fish feast
2 musselback sculpin + 2 glacial salmon + 2 nettlefish + northern spices

2 flask of pure mojo
7 icethorn + 3 pygmy oil + frost lotus + enchanted vial

scroll of enchant weapon - mighty spellpower
30 infinite dust + 6 dream shard + 6 abyss crystal

barbaric bracers
8 heavy leather + 2 cured heavy hide + 4 small lustrous pearl + raptor hide + 4 large fang

earthen leg armor
2 arctic fur + 4 eternal earth + frozen orb

frosthide leg armor
2 arctic fur + 2 nerubian chitin + frozen orb

2 flask of relentless assault
7 terocone + 3 mana thistle + fel lotus + imbued vial

glacial bag
4 moonshroud + 4 ebonweave + eternium thread

#moonshroud
#bolt of imbued frostweave + 2 eternal life

#ebonweave
#bolt of imbued frostweave + 2 eternal shadow

frostweave bag
6 bolt of imbued frostweave + 2 eternium thread

primal might
primal earth + primal water + primal fire + primal air + primal mana

primal earth
10 mote of earth

primal life
10 mote of life

primal water
10 mote of water

primal air
10 mote of air

primal fire
10 mote of fire

primal mana
10 mote of mana

primal shadow
10 mote of shadow

elixir of mighty defense
2 icethorn + crystallized life + imbued vial

practice lock
bronze bar + 2 handful of copper bolts + weak flux

bronze bar
copper bar + tin bar

copper bar
copper ore

tin bar
tin ore

handful of copper bolts
copper bar

glyph of pick lock
lion's ink + light parchment

lion's ink
2 golden pigment

elixir of the mongoose
2 mountain silversage + 2 plaguebloom + crystal vial

elixir of major agility
terocone + 2 felweed + imbued vial

boots of wintry endurance
3 arctic fur + 20 icy dragonscale + 6 runed orb

arctic fur
10 heavy borean leather

heavy borean leather
6 borean leather


"""

r_out = None
for line in recipe_listing.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()


vendor = {
    "wild spineleaf": 0.45,
    "enchanted vial": 0.90,
    "imbued vial": 0.36,
    "leaded vial": 0.016,
    "weak flux": 0.01,
    "light parchment": 0.01,
}


from procurement import ProcurementPlanner
from procurement import topk_procurements
import procurement
from cytoolz import groupby
from cytoolz import valmap
from cytoolz import topk
pp = ProcurementPlanner(items, item_info, r, vendor, approaches=["ah_buy_now", "craft", "vendor"])
pp_market = ProcurementPlanner(items, item_info, r, vendor, approaches=["ah_buy_market", "craft", "vendor"])

def topn_market(item, n=3):
   return topk(
       n,
       [
           (procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(item))
       ],
       lambda x: -procurement.cost(x),
   )

In [12]:
everybody = """
20 indestructible potion
20 potion of speed
20 potion of wild magic
20 flask of stoneblood
20 flask of the frost wyrm
20 flask of pure mojo
20 flask of endless rage
20 skyflare diamond
20 earthsiege diamond
""".strip().splitlines()
best_procurements_market = procurement.And.flat(
    min(procurement.dnf(pp_market.obtain(i(x.strip()))), key=procurement.cost)
    for x in everybody
)
best_procurements_now = procurement.And.flat(
    min(procurement.dnf(pp.obtain(i(x.strip()))), key=procurement.cost)
    for x in everybody
)

In [13]:
best_procurements = best_procurements_now
groupby(
    lambda x: type(x),
    [
        op.sum(v)
        for ((op, name), v)
        in groupby(
            lambda y: (type(y), y.item.pure()[0]),
            best_procurements
        ).items()
    ],
)

{procurement.Craft: [Craft(item=20*indestructible potion, ingredients=40.0*icethorn + 20.0*imbued vial),
  Craft(item=20*potion of speed, ingredients=40.0*adder's tongue + 20.0*pygmy oil + 20.0*imbued vial),
  Craft(item=20*potion of wild magic, ingredients=40.0*lichbloom + 20.0*pygmy oil + 20.0*imbued vial),
  Craft(item=20*flask of stoneblood, ingredients=70.0*lichbloom + 10.0*enchanted vial + 10.0*frost lotus + 30.0*crystallized life),
  Craft(item=20*flask of the frost wyrm, ingredients=50.0*lichbloom + 10.0*enchanted vial + 10.0*frost lotus + 50.0*icethorn),
  Craft(item=20*flask of pure mojo, ingredients=10.0*frost lotus + 10.0*enchanted vial + 30.0*pygmy oil + 70.0*icethorn),
  Craft(item=20*flask of endless rage, ingredients=70.0*lichbloom + 10.0*enchanted vial + 10.0*frost lotus + 30.0*goldclover),
  Craft(item=20*skyflare diamond, ingredients=20.0*bloodstone + 20.0*eternal air + 20.0*chalcedony)],
 procurement.AHBuyNow: [AHBuyNow(item=160.0*icethorn, gold=0.0),
  AHBuyNow(ite

In [14]:
[(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("swiftness potion")))]

[(8.7616, And(AHBuyMarket(item=swiftness potion, gold=8.7616),)),
 (3.2047000000000003,
  And(Craft(item=swiftness potion, ingredients=briarthorn + swiftthistle + leaded vial), AHBuyMarket(item=briarthorn, gold=0.769), AHBuyMarket(item=swiftthistle, gold=2.4197), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016)))]

In [15]:
[(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("swim speed potion")))]

[(0.8443, And(AHBuyMarket(item=swim speed potion, gold=0.8443),)),
 (4.7476,
  And(Craft(item=swim speed potion, ingredients=leaded vial + swiftthistle + blackmouth oil), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016), AHBuyMarket(item=swiftthistle, gold=2.4197), AHBuyMarket(item=blackmouth oil, gold=2.3119))),
 (3.2237,
  And(Craft(item=swim speed potion, ingredients=leaded vial + swiftthistle + blackmouth oil), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016), AHBuyMarket(item=swiftthistle, gold=2.4197), Craft(item=blackmouth oil, ingredients=2.0*oily blackmouth), AHBuyMarket(item=2.0*oily blackmouth, gold=0.788)))]

In [16]:
topk(3, [(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("elixir of water walking")))], lambda x: -procurement.cost(x))

((1.6621, And(AHBuyMarket(item=elixir of water walking, gold=1.6621),)),)

In [17]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("flask of pure mojo")))])

[(12.7961, And(AHBuyMarket(item=flask of pure mojo, gold=12.7961),)),
 (14.696200000000001,
  And(Craft(item=flask of pure mojo, ingredients=1.5*pygmy oil + 0.5*enchanted vial + 0.5*frost lotus + 3.5*icethorn), AHBuyMarket(item=1.5*pygmy oil, gold=3.9699), VendorBuy(item=0.5*enchanted vial, who='vendor', where='somewhere', gold=0.9), AHBuyMarket(item=0.5*frost lotus, gold=3.0678), AHBuyMarket(item=3.5*icethorn, gold=6.7585))),
 (15.39955,
  And(Craft(item=flask of pure mojo, ingredients=1.5*pygmy oil + 0.5*enchanted vial + 0.5*frost lotus + 3.5*icethorn), Craft(item=1.5*pygmy oil, ingredients=1.5*pygmy suckerfish), AHBuyMarket(item=1.5*pygmy suckerfish, gold=4.6732499999999995), VendorBuy(item=0.5*enchanted vial, who='vendor', where='somewhere', gold=0.9), AHBuyMarket(item=0.5*frost lotus, gold=3.0678), AHBuyMarket(item=3.5*icethorn, gold=6.7585))),
 (31.2962,
  And(Craft(item=flask of pure mojo, ingredients=1.5*pygmy oil + 0.5*enchanted vial + 0.5*frost lotus + 3.5*icethorn), AHBuyMar

In [18]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("scroll of enchant weapon - mighty spellpower")))])

[(1196.3076,
  And(Craft(item=scroll of enchant weapon - mighty spellpower, ingredients=6.0*abyss crystal + 30.0*infinite dust + 6.0*dream shard), AHBuyMarket(item=6.0*abyss crystal, gold=1024.5), AHBuyMarket(item=30.0*infinite dust, gold=49.593), AHBuyMarket(item=6.0*dream shard, gold=122.21459999999999))),
 (1577.4546,
  And(AHBuyMarket(item=scroll of enchant weapon - mighty spellpower, gold=1577.4546),))]

In [19]:
hodir_badges_needed = (21000-4425)/50

In [20]:
hodir_badges_needed

331.5

In [21]:
item_info("barbaric bracers")

{'id': 18948,
 'name': 'barbaric bracers',
 'num_auctions': 9999,
 'quantity': 9999,
 'weight_sell': 0.7945,
 'avg_sell': 0.7945,
 'max': 0.7945,
 'p80': 0.7945,
 'p50': 0.7945,
 'p20': 0.7945,
 'wp80': 0.7945,
 'wp50': 0.7945,
 'wp20': 0.7945,
 'min': 0.7945,
 'realm_market_value': 99.0,
 'realm_historical': 237.0,
 'region_historical': 57.4997,
 'region_avg_sale_price': 71.1401,
 'sale_pct': 30.8,
 'sold_per_day': 3,
 'headroom': -9989,
 'market_skew_pct': -58.23,
 'auction_skew_pct': -100.0}

In [22]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("barbaric bracers")))])

[(30.392100000000003,
  And(Craft(item=barbaric bracers, ingredients=4.0*large fang + 2.0*cured heavy hide + 4.0*small lustrous pearl + 8.0*heavy leather + raptor hide), AHBuyMarket(item=4.0*large fang, gold=11.17), AHBuyMarket(item=2.0*cured heavy hide, gold=4.9152), AHBuyMarket(item=4.0*small lustrous pearl, gold=10.8592), AHBuyMarket(item=8.0*heavy leather, gold=3.42), AHBuyMarket(item=raptor hide, gold=0.0277))),
 (99.0, And(AHBuyMarket(item=barbaric bracers, gold=99.0),))]

In [23]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("earthen leg armor")))])

[(171.5737,
  And(Craft(item=earthen leg armor, ingredients=2.0*arctic fur + frozen orb + 4.0*eternal earth), AHBuyMarket(item=2.0*arctic fur, gold=60.1086), AHBuyMarket(item=frozen orb, gold=75.3863), AHBuyMarket(item=4.0*eternal earth, gold=36.0788))),
 (173.7211,
  And(Craft(item=earthen leg armor, ingredients=2.0*arctic fur + frozen orb + 4.0*eternal earth), Craft(item=2.0*arctic fur, ingredients=20.0*heavy borean leather), Craft(item=20.0*heavy borean leather, ingredients=120.0*borean leather), AHBuyMarket(item=120.0*borean leather, gold=62.25600000000001), AHBuyMarket(item=frozen orb, gold=75.3863), AHBuyMarket(item=4.0*eternal earth, gold=36.0788))),
 (238.19910000000002,
  And(Craft(item=earthen leg armor, ingredients=2.0*arctic fur + frozen orb + 4.0*eternal earth), Craft(item=2.0*arctic fur, ingredients=20.0*heavy borean leather), AHBuyMarket(item=20.0*heavy borean leather, gold=126.73400000000001), AHBuyMarket(item=frozen orb, gold=75.3863), AHBuyMarket(item=4.0*eternal eart

In [24]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("frosthide leg armor")))])

[(140.37890000000002,
  And(Craft(item=frosthide leg armor, ingredients=2.0*arctic fur + 2.0*nerubian chitin + frozen orb), AHBuyMarket(item=2.0*arctic fur, gold=60.1086), AHBuyMarket(item=2.0*nerubian chitin, gold=4.884), AHBuyMarket(item=frozen orb, gold=75.3863))),
 (142.52630000000002,
  And(Craft(item=frosthide leg armor, ingredients=2.0*arctic fur + 2.0*nerubian chitin + frozen orb), Craft(item=2.0*arctic fur, ingredients=20.0*heavy borean leather), Craft(item=20.0*heavy borean leather, ingredients=120.0*borean leather), AHBuyMarket(item=120.0*borean leather, gold=62.25600000000001), AHBuyMarket(item=2.0*nerubian chitin, gold=4.884), AHBuyMarket(item=frozen orb, gold=75.3863))),
 (181.3761, And(AHBuyMarket(item=frosthide leg armor, gold=181.3761),)),
 (207.0043,
  And(Craft(item=frosthide leg armor, ingredients=2.0*arctic fur + 2.0*nerubian chitin + frozen orb), Craft(item=2.0*arctic fur, ingredients=20.0*heavy borean leather), AHBuyMarket(item=20.0*heavy borean leather, gold=126

In [25]:
# hateful PVP gear
sum(
[
    4520, # gloves
    7700, # helm
    6110, # spaulders
    7700, # legguards
    7700, # robes
    3990, # idol
]
)


sum(
[
    10850, # belt
    #10850, # boots
    6895, # armwraps
    8295, # pendant
    #8295, # cloak
    #8295, # band
]
)

26040

In [26]:
options = procurement.dnf(pp_market.obtain(i("glacial bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(1410.1313,
 [Craft(item=glacial bag, ingredients=4.0*moonshroud + eternium thread + 4.0*ebonweave),
  AHBuyMarket(item=4.0*moonshroud, gold=703.8332),
  AHBuyMarket(item=eternium thread, gold=5.2465),
  AHBuyMarket(item=4.0*ebonweave, gold=701.0516)])

In [27]:
options = procurement.dnf(pp_market.obtain(i("glacial bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(1410.1313,
 [Craft(item=glacial bag, ingredients=4.0*moonshroud + eternium thread + 4.0*ebonweave),
  AHBuyMarket(item=4.0*moonshroud, gold=703.8332),
  AHBuyMarket(item=eternium thread, gold=5.2465),
  AHBuyMarket(item=4.0*ebonweave, gold=701.0516)])

In [28]:
options = procurement.dnf(pp_market.obtain(i("4 frostweave bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(295.12879999999996,
 [Craft(item=4*frostweave bag, ingredients=24.0*bolt of imbued frostweave + 8.0*eternium thread),
  Craft(item=24.0*bolt of imbued frostweave, ingredients=48.0*infinite dust + 48.0*bolt of frostweave),
  AHBuyMarket(item=48.0*infinite dust, gold=79.3488),
  Craft(item=48.0*bolt of frostweave, ingredients=240.0*frostweave cloth),
  AHBuyMarket(item=240.0*frostweave cloth, gold=173.808),
  AHBuyMarket(item=8.0*eternium thread, gold=41.972)])

In [29]:
options = procurement.dnf(pp_market.obtain(i("primal might")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(46.229400000000005,
 [Craft(item=primal might, ingredients=primal earth + primal fire + primal mana + primal water + primal air),
  AHBuyMarket(item=primal earth, gold=1.1787),
  AHBuyMarket(item=primal fire, gold=3.8232),
  AHBuyMarket(item=primal mana, gold=13.8385),
  Craft(item=primal water, ingredients=10.0*mote of water),
  AHBuyMarket(item=10.0*mote of water, gold=16.283),
  Craft(item=primal air, ingredients=10.0*mote of air),
  AHBuyMarket(item=10.0*mote of air, gold=11.106)])

In [30]:
# drpannkoeken
"Tukui:Profile:fA1smTX1A4X2KeGCrbWxiQ6gPyDlfUcfLIneAuvv8WGnm0Idw2MKI7TPZX2hyg8WCMCMXgiTsXcvv11ErFSd5k1nDtf7URYiu23PSOBdkkBAxDfsTQnTW0ZJXJH0kfVW(4Z57)13)35FMqJ28oaSgOKo06Z2FbdlBGUoWwdzmSukKburkJMv5rnJwOw1AAP3YeI1GgLHsaC8e5sN9V7GeUNxkdQo8th5i2HZuM6pYbtwCpPLkuyPm5wyE5c7S8czbyOHTBVYo3QXBjq(il2DOVyxYcZHyMohQCnRca8Aq75WGnGflXDG7fD7t2PsYt2LzKrJWoccHoQJn(ONn9TlKohb9LiOvIsbj7O96E0f4mAgABamlUtHLYYYicUbKD6w4(Su5rEN89XeecB(pzUDr4Q2Pa2GcWTSLqBvSfptwm9Cu7Ikl(V8oMNnxr4Me7eCpxr3OSJs65n2J888Kf1EYZPH2CaMlZcTd4MI5ANcdj7SyJEpK6kNr9(rIDEpLbpVnGtWfL8J6LRqQRvomeX5hZXKrRmgLbAuRO7LyLT41f(irAzloQWaXOXUFgWttQTHEFZWSk35EcHi)2L5WNc6zmyiEE(GyNVbZG3trGwx8CyreQnUlttszXQoH4DxL(fEcPEEbDXufD7LaKid26Zz8JSyxD9VPfU7f6WHNChohwTr4ybnBPARUQmeubIl6oaV)HJjW7ZVQ3XKfcVqWIx0TpU8X(BdDixZf5)DvIRnh8uL7SalBkv3UuR5eIxQLs(7D0AbEnrBVkk(WFLld6oCfNok5CARPEg9t7sUStxkm8ZjCn6c)SiRoyBioil87qccB5WYIIj)dkJUhpl0HyBRIji6ycSrKfJE8xY8PJ6hCmDXoCUcSwGJQJojgdXcp63OiAXdkaBVDrmXp(cSXKfVPh)sMZ9If(jbAX5XOAM0gDr3X4428784XSQJWUDQJciKUTM5mgLvr4askQNeNKsj0hnpAY5iGwLanZve6MRzs1qGY8hS8cPXye2)QT7i8sLOupK1THIrOYIdAFZ6msBh7yxGlkw3mcl743atbl93iGMic7gJtMh()BeqY3vRcrw0oUovtEssQC0jFsVD)RQSB0w4q0SRWakEnHWh2r3IS4fs8afMTG)Le9SmcCVaozQjcy5)Jq3ESWoP3plgCJRGkgc4cf)0B9R(lpHHlU3p1OZT(ziZBbTJRF5ApH)T(1tYMl6dLlapllAIoH3J)Vp65KumsZ2cAR6eKK5)vSgx5cnNvfGbLTH4XvoxU5RPPxzCLisZJO)2xP8BBzd3yCLEKkOTbCCLl1myO14kx8iIX2SllYAvQanuKOBmCZSilnk3AvVelNMSoMRPI7o2GVBOwlyTSrfu5QWkQ(hKWDSsYqQNEw3dUsOCKgOT6ixHB(u1DNXnkXqxjf3zjSBkzXL98OZdgCLECt)4b5LE84uGJXaQYbY8MB62E6gVmat)YaK4Vaqw89FOFUi7CtbM(Vd(4DY9t5WEOaAfiaT6DNcKMqvnJ1gnr26z7L8v6En7)UiSELmaZ28zqVDKN9vKwAyzXrE2hsuwH3jp02MyT1x721DEABZsBqFvbvZ(xQ06qsSQdzrbIvnhkLMLjr0WVOKfTjetYd1wZuZgLdAc0WQ7txNhQR)21mQQINhAaXa9H33)jX5ld0HF8)WDXFWS)BJWBa0LQzBJmYR9a4nkjJSFh42kMdqhIZpWkleNdT5RygLSh5o)z2nYb(wBcktQJiMdKFtn7YQeOlzWFwQAR8QatyEvTvTvzoEYZ64DykrY(8Wo1lew6Ejo7Ey67jqn4GG8HL(sKPVwdxYN)CV10TyUMjkF8GLweTwgWwPU6ECPpt2wPNCSbYJ6UMYtLj7Ot3DwKoc)MHYEUjU(eVr8iTw2qJpRZ6tAXE1NzQHbwQs0jmQhSSHUMrv)hJOqEtNc8J3JF9M(4tlslIowGFqtQD8L5yBRkrhbnTRUFmNAYiT45hTUQ2Kvb0LFJerFyn6b(nZzriCflLJ8)BBXOsoMZNEpwO5K2RT)S6allTY0bcyLMZqTGkov(Zp"
# trowel
"Tukui:Profile:Tz1smTX1A4X2KecPOa4crv3ibQLYfHIsbdHg1wfpmyddn4GLTjT4EB7CSNdmdEyotpZyEKgPyHQQ6AVOp2HCL6MUPID3vmc19DUS4UnOOSPDvfsTQnTW0ZJXJJtIxyF8589)67)7)mtKrQFxawhuYaA)LhUGPTdWWa4OJmhqkfYeQiLr3U8iwXluTsv90BBbX6qZYqjaEIe5sN9fDqcVZlLbTj8lg(e2HZuM6pYbtwCFPLkuyPm5wyE5c7U8czbyOPJxxYU3Q27iq(il2zKVEpYcR(zMohQCv7ca8AqN5WGnGflXDG3L86w2vn5z7XmYSwuxbHiN0YMa0ZM(ofsNJG(Ye0kXPGKD1FdF6cCgDt9nawf3TWszzzebxVYUDk8jSu5a)Z(Fdjie16LzUDr4QoPaoGcWTDKqBxSbptwm9Cu7Ill(V8pLNnxv4Me7e8oxrV4SJs67)VpW33xwu)HpHgAREzUml0jKBkMRzk0VS7I166yQRChX)Nj25)ig88oaobxukiQxrLuxRCCeIZpLJjJEzmkd0SArVlZkBXRlCFrAzloIqVdrJDpmGpnP2e6NyfLv5UFKqeYVDynWtbTnd6NNN3BOZxJzWhOiqRlEoSic1e3vOjPSyf3i8URspcpKuppJUyQIEDrasKbB)vm(rwSJoEvAH7DHwC4z3LZHvQfDOWMTu1vxvgcuH4IE9Y7F4He495xZ)uYcHNjytu0RBU8X5hICmxZf7)oiX1w99uL7SaBhkv3SuR6gHxQLs(xT0AHEnrtVkk(G)Gld6mQQBlLCo910At)0SKl72Hcd)CcxJUiilYAa2bIdZIGoKGW2USSOyY)MYO7ZZcdi2XUycIoMaByzX4N(nmF6Q9XNsxSlNRaRf6OnrNnedXch8Nuen4bfGD2jSR92((jzmU6G(KApkV7opgv1I2IlIjXlqioQS4w)KppMvCf2RvDuaHmC0TMXSSgchssX9L4KukHUP5rDohb0vdJ(vf6KRzsvtGY8hT8cPXyeoy02ByEPsuQhZ62qXyuzXrnNSAtA76m0f4II1TIXYo(eykyPxGaA8ySjg3mp4xRfsYVNUkrw0mUUvsEgJCCZN0FVNxLDJMchIMDfolEnHOh3s3IS5fYeHkmhHGHeJSmcC)qozQXl6nkNLhvOtFwyN0)3edN4kOHHaUqji9wFWF)HmCt4)l1An1pd5(wqZ4guUoJhm1VEs29Ibq5cW2zrl0z8E8)5GNqsXy1BkOT3KGKC)VQ9ykxO(SAamOSdepMY5YnFvDd1XuIjnpI(B3LYVJTdCJXuUOub9nGJPC56HxAnMYLoHySdByrwxvfAQir3yG6zr26uU1E1sSCAYnXCn1eEJ237hPXc2lBQIkxbQQfCqcVrljdPE6XD23krYrAGoAdFvU5tTP3mEXjg6jP4nlHDtrbDrV0YI3NP2f44Ubf3OmCATWfXlDthnDiaYwue0tjGi45as8CEqw8dFGp9MN(wr29Mcm1DlhorRmtRTmB4R2iuEz31UfiuCfDZ1gjr2nZ2f5R0Dz1Z7HWgQzawnzRWo3Wp(BjnSOYId)4pLOBIUBEOJdXA7VZRJ7(OMML2K(IaAw9SuP1HKyTjKffiwZQ)u62wejbFmilAliMKhAnMPQdkh0cOJ1oKUop0W4DRAwrdpp0eIbgdCyWZzZxgya)SxY72)FREUdcVbWqQQJdYmV(9G3OKmY52WDuS6LEfn)a7SqCo0wVIvCYEKj622n2rbwBbktQJyw9MFlDNYAeOlzYFsPwJ8AalyEn9vD0yoEY2D8UmDgzFEyN6zclDVeTVhM(waudokmFyPVe5Uv7bkfWFE3A6gmxZKC)yFLweTwgW2PgCFUWMjkvVyo21TJ4TQYJKj7yq3DwKbc)wrYEUXV(4V5eXASSPo)Mm7pVb7fBMPkgyRjrV)q7OLnn0nRe8qcfY7XuGF8(8Hx6dhTjTi6qp)G6u74lZX2wtIEbZ0EvcI5utgRbp)O1vL6SkGU87Li6d7rokOzolcHvTvoj4VnfJk5yoF69zHMtAV(HZAaST1lth3Xk1NHAbvCQ8pd"

'Tukui:Profile:Tz1smTX1A4X2KecPOa4crv3ibQLYfHIsbdHg1wfpmyddn4GLTjT4EB7CSNdmdEyotpZyEKgPyHQQ6AVOp2HCL6MUPID3vmc19DUS4UnOOSPDvfsTQnTW0ZJXJJtIxyF8589)67)7)mtKrQFxawhuYaA)LhUGPTdWWa4OJmhqkfYeQiLr3U8iwXluTsv90BBbX6qZYqjaEIe5sN9fDqcVZlLbTj8lg(e2HZuM6pYbtwCFPLkuyPm5wyE5c7U8czbyOPJxxYU3Q27iq(il2zKVEpYcR(zMohQCv7ca8AqN5WGnGflXDG3L86w2vn5z7XmYSwuxbHiN0YMa0ZM(ofsNJG(Ye0kXPGKD1FdF6cCgDt9nawf3TWszzzebxVYUDk8jSu5a)Z(Fdjie16LzUDr4QoPaoGcWTDKqBxSbptwm9Cu7Ill(V8pLNnxv4Me7e8oxrV4SJs67)VpW33xwu)HpHgAREzUml0jKBkMRzk0VS7I166yQRChX)Nj25)ig88oaobxukiQxrLuxRCCeIZpLJjJEzmkd0SArVlZkBXRlCFrAzloIqVdrJDpmGpnP2e6NyfLv5UFKqeYVDynWtbTnd6NNN3BOZxJzWhOiqRlEoSic1e3vOjPSyf3i8URspcpKuppJUyQIEDrasKbB)vm(rwSJoEvAH7DHwC4z3LZHvQfDOWMTu1vxvgcuH4IE9Y7F4He495xZ)uYcHNjytu0RBU8X5hICmxZf7)oiX1w99uL7SaBhkv3SuR6gHxQLs(xT0AHEnrtVkk(G)Gld6mQQBlLCo910At)0SKl72Hcd)CcxJUiilYAa2bIdZIGoKGW2USSOyY)MYO7ZZcdi2XUycIoMaByzX4N(nmF6Q9XNsxSlNRaRf6OnrNnedXch8Nuen4bfGD2jSR92((jzmU6G(KApkV7opgv1I2IlIjXlqioQS4w)KppMvCf2RvDuaHmC0TMXSSgchssX9L4KukHUP5rDo

In [31]:
options = procurement.dnf(pp_market.obtain(i("titanium bar")))
sorted([(procurement.cost(x), x) for x in options], key=lambda x:x[0])

[(9.6154,
  And(Craft(item=titanium bar, ingredients=2.0*titanium ore), AHBuyMarket(item=2.0*titanium ore, gold=9.6154))),
 (12.5425, And(AHBuyMarket(item=titanium bar, gold=12.5425),)),
 (13.064,
  And(Craft(item=titanium bar, ingredients=8.0*saronite bar), Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore), AHBuyMarket(item=16.0*saronite ore, gold=13.064))),
 (13.064,
  And(Craft(item=titanium bar, ingredients=8.0*saronite bar), Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore), AHBuyMarket(item=16.0*saronite ore, gold=13.064))),
 (17.5768,
  And(Craft(item=titanium bar, ingredients=8.0*saronite bar), AHBuyMarket(item=8.0*saronite bar, gold=17.5768)))]

In [32]:
options = procurement.dnf(pp_market.obtain(i("10 elixir of mighty defense")))
sorted([(procurement.cost(x), x) for x in options], key=lambda x:x[0])

[(35.861000000000004,
  And(AHBuyMarket(item=10*elixir of mighty defense, gold=35.861000000000004),)),
 (46.5518,
  And(Craft(item=10*elixir of mighty defense, ingredients=10.0*imbued vial + 20.0*icethorn + 10.0*crystallized life), VendorBuy(item=10.0*imbued vial, who='vendor', where='somewhere', gold=0.36), AHBuyMarket(item=20.0*icethorn, gold=38.620000000000005), Craft(item=10.0*crystallized life, ingredients=eternal life), AHBuyMarket(item=eternal life, gold=7.5718))),
 (48.80200000000001,
  And(Craft(item=10*elixir of mighty defense, ingredients=10.0*imbued vial + 20.0*icethorn + 10.0*crystallized life), VendorBuy(item=10.0*imbued vial, who='vendor', where='somewhere', gold=0.36), AHBuyMarket(item=20.0*icethorn, gold=38.620000000000005), AHBuyMarket(item=10.0*crystallized life, gold=9.822))),
 (48.80200000000001,
  And(Craft(item=10*elixir of mighty defense, ingredients=10.0*imbued vial + 20.0*icethorn + 10.0*crystallized life), VendorBuy(item=10.0*imbued vial, who='vendor', where=

In [33]:
options = procurement.dnf(pp_market.obtain(i("10 practice lock")))
sorted([(procurement.cost(x), x) for x in options], key=lambda x:x[0])

[(30.576999999999998,
  And(Craft(item=10*practice lock, ingredients=20.0*handful of copper bolts + 10.0*weak flux + 10.0*bronze bar), Craft(item=20.0*handful of copper bolts, ingredients=20.0*copper bar), AHBuyMarket(item=20.0*copper bar, gold=11.832), VendorBuy(item=10.0*weak flux, who='vendor', where='somewhere', gold=0.01), AHBuyMarket(item=10.0*bronze bar, gold=18.735))),
 (30.853,
  And(Craft(item=10*practice lock, ingredients=20.0*handful of copper bolts + 10.0*weak flux + 10.0*bronze bar), Craft(item=20.0*handful of copper bolts, ingredients=20.0*copper bar), Craft(item=20.0*copper bar, ingredients=20.0*copper ore), AHBuyMarket(item=20.0*copper ore, gold=12.108), VendorBuy(item=10.0*weak flux, who='vendor', where='somewhere', gold=0.01), AHBuyMarket(item=10.0*bronze bar, gold=18.735))),
 (31.439000000000004,
  And(Craft(item=10*practice lock, ingredients=20.0*handful of copper bolts + 10.0*weak flux + 10.0*bronze bar), Craft(item=20.0*handful of copper bolts, ingredients=20.0*c

In [34]:
options = procurement.dnf(pp_market.obtain(i("glyph of pick lock")))
sorted([(procurement.cost(x), x) for x in options], key=lambda x:x[0])

[(2.0404,
  And(Craft(item=glyph of pick lock, ingredients=lion's ink + light parchment), AHBuyMarket(item=lion's ink, gold=2.0304), VendorBuy(item=light parchment, who='vendor', where='somewhere', gold=0.01))),
 (2.2264,
  And(Craft(item=glyph of pick lock, ingredients=lion's ink + light parchment), Craft(item=lion's ink, ingredients=2.0*golden pigment), AHBuyMarket(item=2.0*golden pigment, gold=2.2164), VendorBuy(item=light parchment, who='vendor', where='somewhere', gold=0.01))),
 (5.815300000000001,
  And(Craft(item=glyph of pick lock, ingredients=lion's ink + light parchment), AHBuyMarket(item=lion's ink, gold=2.0304), AHBuyMarket(item=light parchment, gold=3.7849))),
 (6.0013000000000005,
  And(Craft(item=glyph of pick lock, ingredients=lion's ink + light parchment), Craft(item=lion's ink, ingredients=2.0*golden pigment), AHBuyMarket(item=2.0*golden pigment, gold=2.2164), AHBuyMarket(item=light parchment, gold=3.7849))),
 (12.5618, And(AHBuyMarket(item=glyph of pick lock, gold=12

In [35]:
min([(procurement.cost(x), x) for x in procurement.dnf(pp.obtain(i("55 frost lotus + 87 icethorn + 415 lichbloom")))], key=lambda x:x[0])

(415.624,
 And(AHBuyNow(item=415*lichbloom, gold=0.0), AHBuyNow(item=55*frost lotus, gold=415.624), AHBuyNow(item=87*icethorn, gold=0.0)))

In [36]:
min([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("55 frost lotus + 87 icethorn + 415 lichbloom")))], key=lambda x:x[0])

(1598.5235000000002,
 And(AHBuyMarket(item=415*lichbloom, gold=1093.0685), AHBuyMarket(item=55*frost lotus, gold=337.458), AHBuyMarket(item=87*icethorn, gold=167.997)))

In [37]:
topn_market(i("boots of wintry endurance"), 4)

()